In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# If you need fully deterministic results between runs, set the following environment value prior to launching jupyter.
# See comment in sherlock.features.paragraph_vectors.infer_paragraph_embeddings_features for more info.
%env PYTHONHASHSEED

UsageError: Environment does not have key: PYTHONHASHSEED


# Extract features, retrain Sherlock and generate predictions.

The script below first downloads the data (roughly 700K samples), then extract features from the raw data values. <br>

In [1]:
'''
pyfunctional==1.4.3
-e .
gdown==4.3.0
nltk==3.4.5
gensim==3.8.0
'''

#import sys
#!{sys.executable} -m pip install --user numpy

from datetime import datetime
import os
import sys
import time

import numpy as np
import pandas as pd

from sherlock import helpers
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

Error processing line 1 of /home/sunnykim/miniconda3/envs/env_sherlock/lib/python3.7/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "/home/sunnykim/miniconda3/envs/env_sherlock/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored


In [2]:
print(f'Started at {datetime.now()}.')

Started at 2023-02-24 16:32:11.085308.


## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data for preprocessing and model training into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [3]:
helpers.download_data()

Access denied with the following error:
zip extract11



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1-g0zbKFAXz7zKZc0Dnh74uDBpZCv4YqU 



FileNotFoundError: [Errno 2] No such file or directory: '../data/data.zip'

In [7]:
prepare_feature_extraction()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.


In [8]:
if not os.path.exists('../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy'):
    raise SystemExit(
        """
        Trained paragraph vectors do not exist,
        please run the '03-retrain-paragraph-vector-features' notebook before continuing
        """
    )

## Read in raw data
You can skip this step if you want to use a preprocessed data file.

## Report memory usage (can be slow)

In [9]:
report_memory = False

## Extract features

The input data is assumed to be a dataframe of column values stored as stringed lists "['hello', 'goodbye', 'hi']", stored in a parquet file.

`TODO`: ideally we pickle the dataframe with value lists to avoid parsing these strings into lists again.

In [10]:
# SET FILE PATHS

timestr = time.strftime("%Y%m%d-%H%M%S")

# Features will be output to the following files
X_test_filename_csv = f'../data/data/processed/test_{timestr}.csv'
X_train_filename_csv = f'../data/data/processed/train_{timestr}.csv'
X_validation_filename_csv = f'../data/data/processed/validation_{timestr}.csv'

X_orm_filename_csv = f'../data/data/processed/ormtrain_{timestr}.csv'

## use preprocessed files 
X_test_filename_csv = f'../data/data/processed/test_20220323-163452.csv'
X_train_filename_csv = f'../data/data/processed/train_20220323-163452.csv'
X_validation_filename_csv = f'../data/data/processed/validation_20220323-163452.csv'

### PREPARATION

In [11]:
# ensure embedding initialisation is outside of timing for extract_features
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:08.566848 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:00.017767 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.063516 seconds.


[nltk_data] Downloading package punkt to /home/sunny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sunny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
%load_ext line_profiler

In [13]:
# default process creation mode is changed in Python 3.8 to 'spawn' which causes "name not defined" type errors. Force fork 'mode' for now (this only needs to be called once).
# https://bugs.python.org/issue39931
#mp.set_start_method('fork', force=True)

### Read ORM parquet file

/data/data/raw/df.parquet : data from datadisco & label = data type not col names

/data/data/raw/train_values.parquet etc : online data & label = data col names

In [ ]:
values

In [ ]:

pd.read_parquet("../data/data/raw/labels.parquet")


## EXTRACT FEATURES TO CSV

### TEST SET

In [ ]:
values = load_parquet_values("../data/data/raw/test_values.parquet")

extract_features_to_csv(X_test_filename_csv, values)

values = None

In [ ]:
print(f'Finished at {datetime.now()}')

### TRAIN SET

In [ ]:
values = load_parquet_values("../data/data/raw/train_values.parquet")

extract_features_to_csv(X_train_filename_csv, values)

values = None

In [ ]:
values = load_parquet_values("../data/data/raw/train_values.parquet")

: 

In [ ]:
print(f'Finished at {datetime.now()}')

Finished at 2022-09-29 11:03:10.511524


### VALIDATION SET

In [ ]:
values = load_parquet_values("../data/data/raw/val_values.parquet")

extract_features_to_csv(X_validation_filename_csv, values)

values = None

Starting ../data/data/processed/validation_20220323-163452.csv at 2022-09-29 11:03:11.601911. Rows=137353, using 8 CPU cores
Exporting 1588 column features
Finished. Processed 137353 rows in 0:02:30.577904, key_count=8


In [ ]:
print(f'Finished at {datetime.now()}')

Finished at 2022-09-29 11:05:42.298462


### Read Locally Processed Features

In [ ]:
start = datetime.now()
X_test = pd.read_csv(X_test_filename_csv, dtype=np.float32)

print(f'Load Features (test) process took {datetime.now() - start} seconds.')

Load Features (test) process took 0:00:34.273289 seconds.


In [ ]:
X_test.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.000984,0.000382,-0.000664,0.000514,0.000351,-0.000047,-0.001315,-0.000952,0.000973,-0.000692
1,1.0,0.0,0.368421,0.337950,0.0,2.0,0.0,7.0,0.742677,1.326868,...,0.000823,-0.000610,0.000580,0.000477,0.001203,0.001106,-0.000731,0.000555,-0.000076,0.001190
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.000062,-0.000219,0.000485,0.001242,-0.000786,-0.000344,0.000840,-0.000963,-0.001186,0.000228
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.000080,-0.000734,0.000522,0.000050,-0.000585,-0.000618,-0.000130,-0.000611,0.000368,0.000863
4,1.0,0.0,0.020080,0.035741,0.0,2.0,0.0,5.0,96.521561,9.784149,...,0.001435,0.003560,0.002406,0.002612,0.003109,0.004349,-0.002329,0.002552,-0.003387,-0.001699


In [ ]:
start = datetime.now()

X_train = pd.read_csv(X_train_filename_csv, dtype=np.float32)

print(f'Load Features (train) process took {datetime.now() - start} seconds.')

Load Features (train) process took 0:01:42.935734 seconds.


In [ ]:
X_train.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.000441,0.000100,0.000298,-0.000203,-0.000825,0.000753,-0.001019,0.000132,0.000658,-0.000003
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.001004,0.000121,0.001041,0.000434,0.001836,0.000740,0.000738,0.001208,-0.000341,-0.000308
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.001786,0.002112,0.002798,0.001605,0.003717,0.003941,-0.002605,0.002487,-0.003185,-0.002099
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.001384,-0.000580,0.000207,0.000899,0.000699,0.001308,0.001038,0.001168,-0.000050,0.000921
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.001315,0.000816,-0.000476,-0.001113,0.000889,0.001179,-0.000508,-0.001087,-0.000760,-0.001029


In [ ]:
start = datetime.now()

X_validation = pd.read_csv(X_validation_filename_csv, dtype=np.float32)

print(f'Load Features (validation) process took {datetime.now() - start} seconds.')

Load Features (validation) process took 0:00:34.754495 seconds.


In [ ]:
X_validation.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001025,0.000785,0.000181,-0.001047,0.000341,-0.000907,-0.000261,0.000799,-0.000038,-0.000191
1,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.001562,0.001812,0.001516,-0.000122,0.002120,0.001759,0.000266,0.000366,-0.001418,-0.000433
2,1.0,0.0,0.25000,0.187500,0.0,1.0,0.0,10.0,-0.666667,1.154701,...,-0.000240,0.000450,-0.001238,0.000772,-0.000420,-0.000916,0.001234,0.001150,0.000435,0.000940
3,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.001178,0.000135,-0.000912,-0.000362,0.000512,0.001249,0.000750,0.000857,-0.000996,-0.000050
4,1.0,0.0,0.09434,0.311855,0.0,4.0,0.0,5.0,42.275021,6.530585,...,0.001139,-0.000156,0.000942,-0.000761,-0.000230,-0.000479,-0.001103,0.000289,0.000436,0.000547


## Impute NaN values with feature means

In [ ]:
start = datetime.now()

train_columns_means = pd.DataFrame(X_train.mean()).transpose()

print(f'Transpose process took {datetime.now() - start} seconds.')

Transpose process took 0:00:12.409090 seconds.


In [ ]:
start = datetime.now()

X_train.fillna(train_columns_means.iloc[0], inplace=True)
X_validation.fillna(train_columns_means.iloc[0], inplace=True)
X_test.fillna(train_columns_means.iloc[0], inplace=True)
X_orm.fillna(train_columns_means.iloc[0], inplace=True)

train_columns_means=None

print(f'FillNA process took {datetime.now() - start} seconds.')

FillNA process took 0:00:02.311630 seconds.


In [ ]:
start = datetime.now()

X_train.to_parquet('../data/data/processed/train.parquet', engine='pyarrow', compression='snappy')
X_validation.to_parquet('../data/data/processed/validation.parquet', engine='pyarrow', compression='snappy')
X_test.to_parquet('../data/data/processed/test.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process took {datetime.now() - start} seconds.')

Save parquet process took 0:00:46.659019 seconds.


In [ ]:
print(f'Completed at {datetime.now()}.')

Completed at 2022-09-29 11:09:36.804231.


In [ ]:
X_orm.to_parquet('../data/data/processed/orm.parquet', engine='pyarrow', compression='snappy')